In [11]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import statsmodels.api as sm  

df = pd.read_csv("QSurvey_Synthetic.csv", encoding='ISO-8859-1')

# Rename columns to remove spaces for ease of use
df.rename(columns={
    'DV1 avg': 'DV1_avg',
    'DV2 avg': 'DV2_avg',
    'FL_22_DO': 'Treatment'
}, inplace=True)


In [15]:
# Filter the relevant columns and ensure numeric types
df['DV1_avg'] = pd.to_numeric(df['DV1_avg'], errors='coerce')
df['DV2_avg'] = pd.to_numeric(df['DV2_avg'], errors='coerce')
df['Treatment'] = df['Treatment'].apply(lambda x: 1 if x == 'FL_24' else 0)

In [16]:
# Drop missing values
df_clean = df[['DV1_avg', 'DV2_avg', 'Treatment']].dropna()

# Run regressions
X = sm.add_constant(df_clean['Treatment'])

model_dv1 = sm.OLS(df_clean['DV1_avg'], X).fit()
model_dv2 = sm.OLS(df_clean['DV2_avg'], X).fit()

# Print summaries
print(model_dv1.summary())
print(model_dv2.summary())

# Prepare data for plotting
coef_data = pd.DataFrame({
    'DV': ['DV1_avg', 'DV2_avg'],
    'coef': [model_dv1.params['Treatment'], model_dv2.params['Treatment']],
    'ci_low': [model_dv1.conf_int().loc['Treatment'][0], model_dv2.conf_int().loc['Treatment'][0]],
    'ci_high': [model_dv1.conf_int().loc['Treatment'][1], model_dv2.conf_int().loc['Treatment'][1]]
})

# Plot
plt.figure(figsize=(7, 5))
plt.errorbar(coef_data['DV'], coef_data['coef'], 
             yerr=[coef_data['coef'] - coef_data['ci_low'], coef_data['ci_high'] - coef_data['coef']], 
             fmt='o', capsize=5)
plt.axhline(0, color='grey', linestyle='--')
plt.title('Effect of Treatment on DV1 and DV2')
plt.ylabel('Coefficient Estimate')
plt.xlabel('Dependent Variable')
plt.tight_layout()
plt.show()

ValueError: zero-size array to reduction operation maximum which has no identity

In [10]:
# Add constant to predictors
X = sm.add_constant(df['Treatment'])

# Regression for DV1
model_dv1 = sm.OLS(df['DV1_avg'], X).fit()

# Regression for DV2
model_dv2 = sm.OLS(df['DV2_avg'], X).fit()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [ ]:
#NEW SECTION: SYNTHETIC DVs
#The themes I outlined in my theory are:
# (1) Backlash and Defensive nationalism (2) Normative shifts (3) Trust (4) Distinguishing ppl from gov

#DV1 Attitudes toward the speaker: trust, distinction, norms
## Q10: Do you believe the Canadian Prime Minister's criticism was sincere?
## Q11 Did the message make you feel more favorable toward the Canadian gov?
## Q14 How would you rate yout feelings toward the Canadian prime minister?

#DV2 Attitudes toward their gov: backlash/defensive nationalism, norm, viable audiences
##Q9 I support the US gov
##Q12 I feel more critical of the US gov after reading the article
##Q13 I believe the US should reconsider its current approach.

''' 
Pre Questions:
Q4 My political preferences lean...
o	Very liberal  (1) 
o	Liberal  (2) 
o	Moderate  (3) 
o	Conservative  (4) 
o	Very conservative  (5) 
o	Prefer not to say  (6) 

Q5 I take pride in the U.S. Constitution and feel that its core values align with my personal beliefs.
o	Strongly disagree  (1) 
o	Somewhat disagree  (2) 
o	Neither agree nor disagree  (3) 
o	Somewhat agree  (4) 
o	Strongly agree  (5) 

Q6 I would rather be a citizen of America than any other country.
o	Strongly disagree  (1) 
o	Somewhat disagree  (2) 
o	Neither agree nor disagree  (3) 
o	Somewhat agree  (4) 
o	Strongly agree  (5) 

Q7 It would be disrespectful for myself or another American to criticize U.S. policies.
o	Strongly disagree  (1) 
o	Somewhat disagree  (2) 
o	Neither agree nor disagree  (3) 
o	Somewhat agree  (4) 
o	Strongly agree  (5) 

'''

'''
Q8 At this moment, which government do you feel most morally aligned with?
o	The United States  
o	Canada  
o	Neither  
o	Other (please specify)  

Q9 Indicate your level of agreement:   I felt the need to defend and support the U.S. after reading the Prime Minister's remarks.
o	Strongly disagree  
o	Somewhat disagree  
o	Neither agree nor disagree  
o	Somewhat agree  
o	Strongly agree  

Q10 The Prime Minister's remarks reflect my own concerns and I felt understood, even if I felt the need to defend the U.S.
o	Strongly disagree  
o	Somewhat disagree  
o	Neither agree nor disagree  
o	Somewhat agree  
o	Strongly agree  

Q11 I believe the Prime Minister’s message was sincere.
o	Strongly disagree   
o	Somewhat disagree  
o	Neither agree nor disagree  
o	Somewhat agree  
o	Strongly agree 

Q12 I support the Prime Minister's right to comment on this issue.
o	Strongly disagree  
o	Somewhat disagree  
o	Neither agree nor disagree  
o	Somewhat agree  
o	Strongly agree  

Q13 After reading this article, my view on the Canadian Prime Minister is...
o	Extremely negative  
o	Somewhat negative  
o	Neither positive nor negative  
o	Somewhat positive  
o	Extremely positive  

Q14 After reading this article, my view of the U.S. government is... 
o	Extremely negative  
o	Somewhat negative  
o	Neither positive nor negative  
o	Somewhat negative  
o	Somewhat positive  

Q15 Based on the article you read, how much do you support the U.S. government's data policy?
o	Not at all  
o	Slightly  
o	Somewhat  
o	Mostly  
o	Extremely  
o	Not sure  

'''

